In [1]:
import time
import glob
import torch
import torch.optim as optim
import os

print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.4.0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16280MB, multi_processor_count=56)


In [2]:
from google.colab import drive
drive.mount('/content/drive/')  # 将google硬盘挂载在/comtent/drive/目录上面

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
path  = '/content/drive/My Drive/Colab Notebooks/glasses_classification'
os.chdir(path)
from mobilenetv2 import mobilenetv2

In [4]:
#####build the network model
model = mobilenetv2(num_classes=2, width_mult=0.1)

#打印模型架构
print(model)

MobileNetV2(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 4, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=4, bias=False)
        (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(4, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(4, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv

In [0]:
#a = torch.rand([1,3,64,64])
#outputs = model(a)
#print(outputs)

tensor([[-0.6890, -0.1649]], grad_fn=<AddmmBackward>)


In [5]:
###load the pretrained weights，下载的预训练模型
pretrained_path = 'weights/mobilenetv2_0.1-7d1d638a.pth'

print("Initializing the network ...")
#读入预训练模型的参数
#pytorch存储的模型一般采取存储网络参数格式，保存的格式为字典的格式，键为定义每层op的名字，值为保存的参数值

state_dict = torch.load(pretrained_path)

###去掉全链接层的权重，
#由于我们一般不会直接使用，imagenet的1000类，因此，我们需要更换网络最后的全链接层
#因此我们需要将前边几层的参数保存，最后一层重新初始化
#定义一个新的字典，将原始的参数字典，对应保存与更改
from collections import OrderedDict
new_state_dict = OrderedDict()

for k,v in state_dict.items():
    #print(k)  #打印预训练模型的键，如果发现与网络定义的键有一定的差别，可以更改对应的键值，
    #torchvision中的网络定义，采用了正则表达式，来更改键值，因为这里简单，没有再去构建正则表达式
    if k.split('.')[0] == 'features' and int(k.split('.')[1]) > 10:#该模型只取了前10个InvertedResidual
        continue
    ##得到新的与定义网络对应的预训练参数
    new_state_dict[k] = v
##导入网络参数
model.load_state_dict(new_state_dict)

Initializing the network ...
features.0.0.weight
features.0.1.weight
features.0.1.bias
features.0.1.running_mean
features.0.1.running_var
features.0.1.num_batches_tracked
features.1.conv.0.weight
features.1.conv.1.weight
features.1.conv.1.bias
features.1.conv.1.running_mean
features.1.conv.1.running_var
features.1.conv.1.num_batches_tracked
features.1.conv.3.weight
features.1.conv.4.weight
features.1.conv.4.bias
features.1.conv.4.running_mean
features.1.conv.4.running_var
features.1.conv.4.num_batches_tracked
features.2.conv.0.weight
features.2.conv.1.weight
features.2.conv.1.bias
features.2.conv.1.running_mean
features.2.conv.1.running_var
features.2.conv.1.num_batches_tracked
features.2.conv.3.weight
features.2.conv.4.weight
features.2.conv.4.bias
features.2.conv.4.running_mean
features.2.conv.4.running_var
features.2.conv.4.num_batches_tracked
features.2.conv.6.weight
features.2.conv.7.weight
features.2.conv.7.bias
features.2.conv.7.running_mean
features.2.conv.7.running_var
feature